# The lack of PFS: a danger to privacy

In [ ]:
from scapy.all import *
load_layer('tls')

In [ ]:
record1_str = open('raw_data/tls_session_compromised/01_cli.raw', 'rb').read()
record1 = TLS(record1_str)
record1.msg[0].show()

In [ ]:
record2_str = open('raw_data/tls_session_compromised/02_srv.raw', 'rb').read()
record2 = TLS(record2_str, tls_session=record1.tls_session.mirror())
record2.msg[0].show()

In [ ]:
# Suppose we possess the private key of the server
# Try registering it to the session
#key = PrivKey('raw_data/pki/srv_key.pem')
#record2.tls_session.server_rsa_key = key

In [ ]:
record3_str = open('raw_data/tls_session_compromised/03_cli.raw', 'rb').read()
record3 = TLS(record3_str, tls_session=record2.tls_session.mirror())
record3.show()

In [ ]:
record4_str = open('raw_data/tls_session_compromised/04_srv.raw', 'rb').read()
record4 = TLS(record4_str, tls_session=record3.tls_session.mirror())
record4.show()

In [ ]:
record5_str = open('raw_data/tls_session_compromised/05_cli.raw', 'rb').read()
record5 = TLS(record5_str, tls_session=record4.tls_session.mirror())
record5.show()